In [ ]:
import torch
import numpy as np

In [ ]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

In [ ]:
# for downloading the data and unziping it
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2024-03-24 15:44:50--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.170.207, 173.194.174.207, 74.125.23.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.170.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  21.8MB/s    in 2m 56s  

2024-03-24 15:47:46 (20.7 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [ ]:
classes = ["Amphibia", "Animalia", "Arachnida", "Aves", "Fungi", "Insecta", "Mammalia", "Mollusca", "Plantae", "Reptilia"]


In [ ]:
import os

def count_images(root_dir):
    for dir_name, _, file_list in os.walk(root_dir):
        print(f"{dir_name}: {len(file_list)} images")

# Assuming the base directory is 'inaturalist_12K'
base_dir = 'inaturalist_12K'

count_images(os.path.join(base_dir, 'train'))
count_images(os.path.join(base_dir, 'val'))



inaturalist_12K/train: 1 images
inaturalist_12K/train/Amphibia: 1000 images
inaturalist_12K/train/Arachnida: 1000 images
inaturalist_12K/train/Mammalia: 1000 images
inaturalist_12K/train/Mollusca: 1000 images
inaturalist_12K/train/Reptilia: 1000 images
inaturalist_12K/train/Insecta: 1000 images
inaturalist_12K/train/Animalia: 1000 images
inaturalist_12K/train/Aves: 1000 images
inaturalist_12K/train/Plantae: 1000 images
inaturalist_12K/train/Fungi: 1000 images
inaturalist_12K/val: 0 images
inaturalist_12K/val/Amphibia: 200 images
inaturalist_12K/val/Arachnida: 200 images
inaturalist_12K/val/Mammalia: 200 images
inaturalist_12K/val/Mollusca: 200 images
inaturalist_12K/val/Reptilia: 200 images
inaturalist_12K/val/Insecta: 200 images
inaturalist_12K/val/Animalia: 200 images
inaturalist_12K/val/Aves: 200 images
inaturalist_12K/val/Plantae: 200 images
inaturalist_12K/val/Fungi: 200 images
